In [1]:
# Librerias necesarias para procesamiento y entrenamiento
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd
import pickle
import pygame

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mp_drawing = mp.solutions.drawing_utils   #Ayudas para dibujar
mp_holistic = mp.solutions.holistic       #Soluciones de Mediapipe

In [3]:
# Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()      
        
        # Reformatear: BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el modelo holistico:
        results = holistic.process(image)               
        
        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Mostramos la imagen                
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [4]:
results.face_landmarks.landmark[0]

x: 0.568193614
y: 0.54444766
z: -0.040950533

In [5]:
print("x:", results.pose_landmarks.landmark[0].x,"y:", 
      results.pose_landmarks.landmark[0].y,"z:", 
      results.pose_landmarks.landmark[0].z)

x: 0.5698683261871338 y: 0.44117605686187744 z: -1.3352296352386475


In [6]:
#Tenemos 501 coordenadas en total
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [7]:
#la primera columna servirá de identificación de clase 
landmarks = ['class']

for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

#Mostramos ejemplo:
print(landmarks[0:13])

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3', 'y3', 'z3', 'v3']


In [8]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [9]:
class_name = "Cansado"

In [10]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [11]:
class_name = "Despierto"

In [12]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [13]:
# Leemos el archivo .csv
df = pd.read_csv("coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Cansado,0.544687,0.395883,-1.539087,0.999957,0.583237,0.316030,-1.453993,0.999895,0.606122,...,-0.007612,0.0,0.644743,0.326947,0.024440,0.0,0.651243,0.319448,0.025341,0.0
1,Cansado,0.540882,0.396399,-1.521091,0.999959,0.577496,0.316047,-1.428508,0.999900,0.600003,...,-0.006900,0.0,0.644513,0.329228,0.025394,0.0,0.651081,0.322172,0.026193,0.0
2,Cansado,0.536408,0.396965,-1.501423,0.999960,0.573696,0.316206,-1.404768,0.999903,0.596322,...,-0.006164,0.0,0.640682,0.325094,0.026010,0.0,0.646997,0.318728,0.026719,0.0
3,Cansado,0.529686,0.398477,-1.504289,0.999961,0.568217,0.317275,-1.409980,0.999903,0.591150,...,-0.006963,0.0,0.640738,0.327853,0.025599,0.0,0.647201,0.321395,0.026304,0.0
4,Cansado,0.525401,0.401187,-1.506657,0.999959,0.564754,0.318983,-1.413395,0.999899,0.587947,...,-0.009340,0.0,0.639545,0.336558,0.022728,0.0,0.645909,0.331068,0.023257,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,Despierto,0.532796,0.414148,-1.345027,0.999979,0.572854,0.323185,-1.263658,0.999947,0.599495,...,-0.006839,0.0,0.658856,0.354467,0.027909,0.0,0.665883,0.347335,0.028767,0.0
249,Despierto,0.532794,0.415689,-1.342008,0.999980,0.572868,0.324015,-1.260755,0.999950,0.599515,...,-0.007155,0.0,0.659818,0.353606,0.027092,0.0,0.666767,0.346916,0.027852,0.0
250,Despierto,0.533559,0.416404,-1.324970,0.999980,0.573075,0.324524,-1.249209,0.999948,0.599607,...,-0.007477,0.0,0.661962,0.366005,0.025662,0.0,0.669417,0.357271,0.026676,0.0
251,Despierto,0.534213,0.417591,-1.324299,0.999978,0.573325,0.325261,-1.249894,0.999944,0.599784,...,-0.008427,0.0,0.663788,0.359358,0.025903,0.0,0.670868,0.352373,0.026726,0.0


In [14]:
# Guardaremos las variables en X y el target en Y
X = df.drop('class',axis=1)
y = df['class']

In [15]:
# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1000)

In [16]:
# Definimos los cuatro pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [17]:
# Guardamos los modelos en la biblioteca fit_models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [18]:
# Probamos a predecir X_test con el modelo de Random Forest
fit_models['rf'].predict(X_test)

array(['Cansado', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Cansado', 'Despierto', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Cansado',
       'Cansado', 'Despierto', 'Cansado', 'Cansado', 'Despierto',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Cansado', 'Despierto', 'Cansado', 'Despi

In [19]:
#Predecir las clases de X_test
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    
    #Mostramos las precisiones de cada modelo
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [20]:
# Escribimos el modelo en un archivo .pkl
with open('cansado_despierto.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [21]:
# Abrimos de nuevo el modelo
with open('cansado_despierto.pkl', 'rb') as f:
    model = pickle.load(f)

In [22]:
# Inicializar pygame mixer
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound('alarma.wav')  # Asegúrate de que el archivo de sonido esté en formato .wav

In [ ]:
# Conectamos la cámara
cap = cv2.VideoCapture(0)

# Inicializar la variable de control de la alarma
alarm_playing = False

# Inicia el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()

        # Verificar que la imagen se haya leído correctamente
        if not ret:
            print("No se pudo recibir imagen de la cámara. Saliendo...")
            break

        # BGR -> RGB y seguridad de la imagen
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detecciones
        results = holistic.process(image)

        # RGB -> BGR y seguridad de la imagen
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Dibujo de landmarks
        # 1. Face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )

        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

        # Exporta coordenadas
        try:
            # Verificar que se hayan detectado landmarks de pose y rostro
            if results.pose_landmarks and results.face_landmarks:
                # Extraemos Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose
                ]).flatten())

                # Extraemos Face landmarks
                face = results.face_landmarks.landmark
                face_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face
                ]).flatten())

                # Concatenamos las filas
                row = pose_row + face_row

                # Hacemos detecciones
                X = pd.DataFrame([row])

                # Predecir la clase
                body_language_class = model.predict(X)[0]

                # Predecir la probabilidad de la clase
                body_language_prob = model.predict_proba(X)[0]

                # Calcular la probabilidad
                probability = round(body_language_prob[np.argmax(body_language_prob)], 2)

                # Mostrar valores para depuración
                print(f"Clase predicha: '{body_language_class}'")
                print(f"Probabilidad: {probability}")

                # Recogemos coordenadas de la oreja izquierda para visualizar texto dinámicamente
                coords = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                    ), [640,480]).astype(int))

                # Mostramos la clase en la oreja izquierda
                cv2.rectangle(
                    image,
                    (coords[0], coords[1]+5),
                    (coords[0]+len(body_language_class)*20, coords[1]-30),
                    (245, 117, 16), -1)
                cv2.putText(
                    image, body_language_class, coords,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Status Box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Mostramos Class
                cv2.putText(
                    image, 'CLASS', (95,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, body_language_class.split(' ')[0], (90,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Mostramos Probabilidad
                cv2.putText(
                    image, 'PROB', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, str(probability), (10,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Al estar muy cansado, activamos la alerta de descanso y la alarma
                if (probability > 0.70) and ("Cansado" in body_language_class):

                    # Mostrar mensaje de alerta
                    cv2.rectangle(
                        image,
                        (coords[0], coords[1]+5),
                        (coords[0]+len('MUY CANSADO')*20, coords[1]-30),
                        (0, 0, 255), -1)

                    cv2.putText(
                        image, 'MUY CANSADO', coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    cv2.rectangle(image, (0,0), (550, 60), (0, 0, 255), -1)

                    cv2.putText(
                        image, "ALERTA! NECESITAS DESCANSAR", (15,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    # Activar alarma si no está sonando
                    if not alarm_playing:
                        alarm_playing = True
                        alarm_sound.play(-1)  # Reproducir en bucle
                else:
                    if alarm_playing:
                        alarm_playing = False
                        alarm_sound.stop()
            else:
                print("No se detectaron landmarks de pose o rostro.")
                # Puedes manejar este caso si es necesario

        except Exception as e:
            print(f"Ocurrió un error en el procesamiento: {e}")
            pass

        # Mostrar la imagen
        cv2.imshow('DETECTOR DE SOMNOLENCIA', image)

        # Salir con la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar ventanas
    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se detectaron landmarks de pose o rostro.
No se dete